# Aritmética de ponto flutuante em computadores

## $ \S 1 $ Introdução

O principal objetivo dos métodos numéricos é desenvolver algoritmos e técnicas
eficazes para resolver problemas matemáticos e, no processo, ajudar-nos a obter
um maior entendimento sobre eles. Os métodos numéricos são especialmente úteis
quando soluções analíticas ou em forma fechada para problemas não estão
disponíveis ou são muito difíceis de se calcular.  Números são a base de toda
computação. Portanto é necessário que primeiro compreendamos como eles são
representados em uma máquina, como sua capacidade de memória finita e poder de
processamento inevitavelmente levam a erros, e como estes erros podem ser
controlados.

## $ \S 2 $ O sistema de representação de números de ponto fixo

No sistema dos __números de ponto fixo__, especifica-se uma base $ b \ge 2 $,
digamos a decimal, e uma quantidade fixa de dígitos para se representar a parte
inteira e a parte fracionária de um número qualquer, digamos $ 3 $ e $ 4 $
respectivamente. Poderíamos portanto representar os números
\begin{align*}
\pi& \quad \text{como}& \ 3&.1416 \\
-\frac{\pi}{1000}& \quad \text{como}& \ -0&.0031 \\
123 & \quad \text{como}& \ 123&.0 \\
-123.456789& \quad \text{como}& \ -123&.4568\,.
\end{align*}
Observe que a diferença entre dois números sucessivos é sempre a
mesma ($ 0.0001 $ no exemplo).

Como veremos, este sistema é muito inflexível e ineficiente para ser utilizado
na prática. Usando base $ b = 2 $ e $ 32 $ bits para armazenar cada número, com
$ 16 $ bits reservados para a parte inteira e $ 16 $ para a parte fracionária,
só seria possível representar números entre $ 2^{-16} \approx 0,00002 $ e $
2^{16} = 65536 $. 

## $ \S 3 $ O sistema de representação de números de ponto flutuante

### $ 3.1 $ Definição dos números de ponto flutuante

A aritmética de ponto flutuante é um pilar dos métodos numéricos, fornecendo um
meio eficiente de representar e manipular números reais em hardware e software.
Os números de ponto flutuante são números expressos num formato específico, que
nos permite representar tanto magnitudes grandes quanto pequenas mantendo
precisão razoável. O padrão [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754)
(estabelecido em 1985), que define os formatos de números de ponto flutuante
utilizados comumente, tornou-se parte integrante da engenharia dos sistemas
modernos de computação e de algoritmos numéricos. Em particular, ele também é
seguido em Python.

Um número de ponto flutuante consiste do produto de duas componentes: uma
__mantissa__ (ou _significando_) e uma __base__ elevada a um __expoente__, por
exemplo:
$$
-1234.56 = -1.23456
\times 10^3 \,.
$$
Mais geralmente, um __número de ponto flutante__ tem a forma
$$
x = \text{(sinal)}\, \, \text{mantissa}\, \times \,\text{base}^{\text{expoente}}\,.
$$
Devem ser respeitadas as seguintes restrições sobre estes elementos:
* O _sinal_ $ \pm $ é representado por um único bit.
* A _base_ $ b $ pode ser qualquer inteiro $ \ge 2 $ (ou seja), mas esta escolha é fixa.
  Na maioria esmagadora dos casos, é utilizado o sistema binário ($ b = 2 $).
* O _expoente_ é representado utilizando-se um número fixo de bits, por exemplo
  $ 8 $ para precisão simples. Dentre as $ 2^8 = 256 $
  possibilidades, duas sempre são reservadas para denotar valores especiais,
  como infinito (`inf` em Python) ou _NaN_ (_Not a Number_, `NaN`). Sendo assim,
  no caso da precisão simples, os possíveis expoentes são aqueles entre $ -126 $
  e $ +127 $. Esta restrição limita as magnitudes dos números que podem ser
  representados precisamente pela máquina. Se o expoente de um número não está
  dentro desta faixa, o resultado pode ser um erro de __overflow__ (muito grande
  para ser representado) ou de __underflow__ (muito pequeno para ser
  representado).
* A _mantissa_ ou _significando_ deve estar no intervalo $ [1, b) $,
  onde $ b $ é a base. Sendo assim, neste formato a mantissa de um número
  envolve exatamente um dígito antes do ponto (e este dígito não pode ser nulo).
  Por exemplo, quando $ b = 10 $ é a base, $ 4.321 $ é um mantissa válida, mas $
  0.4321 $ e $ 43.21 $ não são. A mantissa é representada no sistema de precisão
  simples através de $ 23 $ bits. Na verdade quando $ b = 2 $ ganhamos um bit
  extra de precisão, já que como a mantissa deve estar em $ [1, 2) $, o dígito
  antes do ponto só pode ser $ 1 $; este bit é chamado de _bit escondido_ ou 
  _implícito_. Seja como for, a limitação do número de bits para armazenagem
  da mantissa impõe uma restrição à precisão com que um número pode ser
  representado.
  
No caso do formato de _precisão simples_ são utilizados portanto
$$
32 \text{ bits} = 1 \text{ bit para o sinal} +
8 \text{ bits para o expoente} + 23\text{ bits para a mantissa}
$$
para se representar um número de ponto flutuante. 

📝 O padrão IEEE 754 também contém especificações para representação de números
com dupla ($ 64 $ bits), quádrupla ($ 128 $ bits) e meia ($ 16 $ bits) precisão,
dentre outros.

📝 O termo "ponto flutuante" se refere ao fato que a posição do ponto de base
não é fixa. À medida que o expoente muda, o ponto pode "flutuar" entre
diferentes posições, permitindo que a representação cubra eficientemente uma
ampla gama de números. Python representa os números de ponto flutuante com o
tipo `float`; a sua representação mais fiel é aquela da chamada
_notação científica_: `<mantissa>e<expoente>` (em que a base $ b = 10 $). Por
exemplo $ 123.45 $ corresponde nesta notação a `1.2345e2`.

### $ 3.2 $ Características do sistema de números de ponto flutuante

<div class="alert alert-info">  Para tornar a discussão mais simples e concreta,
de agora em diante assumiremos que no nosso sistema de números de ponto
flutuante a base $ b = 10 $, os expoentes variam de $ -9 $ a $ 9 $ e as
mantissas possuem $ 3 $ dígitos significativos.
</div>

Neste caso:
* O zero é representado por $$ 0 = 0.00 \times 10^{-9}\,. $$ Este é o único
  número cuja mantissa não respeita as restrições descritas na $ \S 2.1 $.
  Como veremos, _não se deve representar o zero por_ $ 0.00 \times 10^{e} $
  _para qualquer expoente_ $ e \ne -9 $, sob pena de termos $ x + 0 = 0 $
  para alguns números $ x \ne 0 $.
* O maior número representável neste sistema é $ 9.99 \times 10^9 $
  (aproximadamente $ 10 $ bilhões), e o menor número positivo é
  $ 1.00 \times 10^{-9} $.
* Números reais de valor absoluto entre $ 0 $ e $ 9.99 \times 10^9 $ são
  __arredondados__ ao número representável mais próximo, seguindo a convenção
  que, por exemplo, $ 1.235 $ é arredondado "para cima", a $ 1.24 \times 10^0 $.
* Chamamos de __década__ o conjunto de números que têm um mesmo expoente $ e $.
  Em cada década do nosso sistema, podem ser representados $ 900 $ números:
  aqueles com mantissa entre $ 1.00 $ e $ 9.99 $.
* Dentro da década de expoente $ e $, números sucessivos estão igualmente
  espaçados por $ 10^e $. Contudo esta distância aumenta exponencialmente
  conforme $ e $ aumenta.

__Exercício:__ Mostre que o nosso sistema consiste de exatamente $ 34\,201 $
números diferentes.

### $ 3.3 $ Limitações da representação de números reais como números de ponto flutuante

A precisão finita inerente à representação de números reais utilizando um
número limitado de dígitos não apenas restringe a classe de números que
podem ser representados de maneira exata, mas também traz inevitavelmente a
possibilidade de erros mesmo nas operações aritméticas mais simples.

Por sua vez, o acúmulo de erros de pode resultar em perda de significância ou
resultados incorretos em certas operações numéricas. Ademais, a aritmética de
ponto flutuante está sujeita ao _cancelamento catastrófico_, um fenômeno em que
a subtração de dois números quase iguais produz um resultado com precisão
significativamente reduzida. Estas limitações exigem uma cuidadosa consideração
da propagação de erros e da estabilidade dos métodos numéricos, bem como a
adoção de técnicas para minimizar o impacto dos erros na precisão e
confiabilidade dos resultados computacionais.


## $ \S 3 $ Adição no sistema de ponto flutuante

__Exemplo 1:__ Vamos somar os números $ 3.15 \times 10^2 $ e $ 1.26 \times 10^1
$.  Antes de realizar a adição, precisamos garantir que os expoentes sejam
iguais. Para isto, vamos "deslocar o ponto" do número com o _menor_ expoente
para que passe a ter o mesmo expoente que o outro número:
\begin{equation*}
    1.26 \times 10^1 = 0.126 \times 10^2 \quad \text{(igualando os expoentes)}
\end{equation*}
Agora podemos realizar a adição:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 3&.15 \times 10^2 \\
        &+& 0&.126 \times 10^2 & \text{(adicionando)} \\
        &=& 3&.276 \times 10^2 & \text{(arredondando)} \\
        &\approx & 3&.28 \times 10^2 & \text{(resultado)}
    \end{aligned}
\end{equation*}
Observe que no último passo realizamos um _arredondamento_ (não truncamento)
para que o resultado tivesse apenas $ 3 $ dígitos significativos na mantissa,
conforme nossa convenção.

__Exemplo 2:__ Calcule a soma dos números $ 6.21 \times 10^{2} $ e
$ 5.23 \times 10^{0} $. Mais uma vez, precisamos igualar os expoentes antes de
realizar a adição. Para isto ajustamos o número com o _menor_ expoente:
\begin{equation*}
    5.23 \times 10^0 = 0.00523 \times 10^2 \quad \text{(igualando os expoentes)}
\end{equation*}
Calculando a soma:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 6&.21 \times 10^2 \\
        &+& 0&.00523 \times 10^2 & \text{(adicionando)}\\
        &=& 6&.21523 \times 10^2 & \text{(arredondando)} \\
        &\approx & 6&.22 \times 10^2 & \text{(resultado)}
    \end{aligned}
\end{equation*}
Observe novamente o uso de arredondamento no último passo para trazer a
mantissa do resultado de volta à nossa convenção.

__Exemplo 3__: Vamos somar os números $ 9.53 \times 10^{-1} $ e
$ 7.48 \times 10^{-2} $. Igualando o expoente do segundo número ao
primeiro:
\begin{equation*}
    7.48 \times 10^{-2} = 0.748 \times 10^{-1} \quad \text{(igualando os expoentes)}
\end{equation*}
Realizando a adição:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 9&.53 &\times& 10^{0} \\
        &+& 0&.748 &\times& 10^{0} & \text{(adicionando)}\\
        &=& 10&.278 &\times& 10^{0} & \text{(renormalizando)} \\
        &=& 1&.0278 &\times& 10^{1} &  \text{(arredondando)}\\
        &\approx& 1&.03 &\times& 10^{1} & \text{(resultado)} \\
    \end{aligned}
\end{equation*}


__Exercício:__ Calcule as somas dos números de ponto flutuante abaixo. Suas
respostas devem seguir a convenção acima, ou seja, os expoentes devem estar entre
$ 9 $ e $ -9 $ e as mantissas devem possuir exatamente três dígitos
significativos.

(a) $ 2.34 \times 10^2 + 3.87 \times 10^2 $.

(b) $ 4.23 \times 10^{-4} + 1.38 \times 10^{-3} $.

(c) $ 3.24 \times 10^5 + 9.78 \times 10^3 $.

(d) $ 7.36 \times 10^{-7} + 4.29 \times 10^{-9} $.

(e) $ 5.99 \times 10^9 + 8.43 \times 10^{-1} $.

(f) $ 1.03 \times 10^9 + 8.97 \times 10^{9} $.

## $ \S 4 $ Subtração no sistema de ponto flutuante


__Exemplo 1__: Vamos subtrair os números $1.52 \times 10^3$ e $7.38 \times
10^2$. Igualando os expoentes:
\begin{equation*}
    7.38 \times 10^{2} = 0.738 \times 10^{3} \quad \text{(igualando os expoentes)}
\end{equation*}
Calculando a diferença:
\begin{equation*}
    \begin{aligned}
        &\phantom{-}& 1&.52 &\times& 10^{3} \\
        &-& 0&.738 &\times& 10^{3} & \text{(subtraindo)}\\
        &=& 0&.782 &\times& 10^{3} & \text{(renormalizando)}\\
        &=& 7&.82 &\times& 10^{2} & \text{(resultado)} \\
    \end{aligned}
\end{equation*}

__Exemplo 2__: Vamos subtrair os números $6.42 \times 10^{6}$ e $4.21 \times
10^{4}$. Igualando o expoente do segundo número ao primeiro:
\begin{equation*}
    4.21 \times 10^4 = 0.000421 \times 10^6 \quad \text{(igualando os expoentes)}
\end{equation*}
Realizando a subtração:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 6&.420 &\times& 10^{6} \\
        &-& 0&.000421 &\times& 10^{6} & \text{(subtraindo)}\\
        &=& 6&.419579 &\times& 10^{6} &  \text{(arredondando)}\\
        &\approx& 6&.420 &\times& 10^{6} &  \text{(resultado)}\\
    \end{aligned}
\end{equation*}

__Exemplo 3__: Vamos subtrair os números $1.93 \times 10^{-4}$ e $9.76 \times 10^{-3}$.
Primeiramente precisamos igualar os expoentes:
\begin{equation*}
    1.93 \times 10^{-4} = 0.193 \times 10^{-3} \quad \text{(igualando os expoentes)}
\end{equation*}
Como o segundo número (subtraendo) $ b $ é maior que o primeiro (minuendo) $ a
$, é mais fácil calcular $ b - a $ e utilizar que $ a - b = -(b - a) $.
Fazendo isto, obtemos:
\begin{equation*}
    \begin{aligned}
        &\phantom{-}& 9&.76 &\times& 10^{-3} \\
        &-& 0&.193 &\times& 10^{-3} & \text{(subtraindo)}\\
        &=& 9&.567 &\times& 10^{-3} & \text{(arredondando)}  \\
        &\approx& 9&.57 &\times& 10^{-3} & \text{(resultado)} \\
    \end{aligned}
\end{equation*}
Assim, a diferença de $1.93 \times 10^{-4}$ e $9.76 \times 10^{-3}$
é aproximadamente $ -9.57 \times 10^{-3} $.



__Exercício:__ Calcule as diferenças abaixo. Suas respostas devem seguir a
convenção acima, ou seja, os expoentes devem estar entre $ 9 $ e $ -9 $ e as
mantissas devem possuir exatamente três dígitos significativos.

(a) $ 6.71 \times 10^1 - 3.42 \times 10^0 $.

(b) $ 9.12 \times 10^{-3} - 2.46 \times 10^{-2} $.

(c) $ 1.98 \times 10^6 - 3.24 \times 10^4 $.

(d) $ 7.25 \times 10^{-7} - 5.93 \times 10^{-5} $.

(e) $ 4.73 \times 10^8 - 3.92 \times 10^8 $.

(f) $ 1.01 \times 10^{-9} - 1.32 \times 10^{-9} $.

__Exercício:__ Mostre que no nosso sistema:

(a) Se $ \vert \delta \vert < \frac{1}{2} \times 10^{-3} $, então
$ 1 \pm \delta = 1 $.

(b) Se $ \varepsilon^2 < \frac{1}{2} \times 10^{-3} $,
então
$$
1 - (1 - \varepsilon)(1 + \varepsilon) = 0\,.
$$